# COVID-19 Data Analysis Using PySpark

This notebook was created as a part of Big Data Mini Project

---

**Problem Statement:** To perform exploratory data analysis on COVID-19 dataset using Apache Spark

**Technologies Used:** Python, Pyspark, Plotly, Numpy

**Dataset Information:** The dataset used is a volunteer-driven, crowd-sourced database for COVID-19 stats & patient tracing in India. It contains state-wise information about the number of people tested for COVID-19 on a daily basis and the data is divided across categories like confirmed, recovered, deceased and active.

**Dataset Link:** https://api.covid19india.org/


In [ ]:
%%capture
!pip install static-frame pyspark 
!pip install --upgrade plotly

In [ ]:
import json
import requests
import static_frame
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pyspark
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

In [ ]:
def get_api_data(url):
    response = requests.get(url)
    if response.status_code != 200:
        if response.status_code == 404:
            print("Data not found!")
            return None
        else:
            raise Exception(f"API Hit Failed - {response.text}")
    return response.json()

In [ ]:
def to_long(df, by):
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))
    assert len(set(dtypes)) == 1, "Columns have to be of the same type"
    kvs = F.explode(
        F.array([F.struct(F.lit(c).alias("key"), F.col(c).alias("val")) for c in cols])
    ).alias("kvs")
    return df.select(by + [kvs]).select(by + ["kvs.key", "kvs.val"])

In [ ]:
# spark session
spark = SparkSession.builder.appName("COVID19-Data-Analysis").getOrCreate()

In [ ]:
api_urls = {
    "state_daily": "https://api.covid19india.org/states_daily.json",
    "state_total": "https://api.covid19india.org/v4/data.json",
}

state_codes = {
    "an": "Andaman and Nicobar Islands",
    "ap": "Andhra Pradesh",
    "ar": "Arunachal Pradesh",
    "as": "Assam",
    "br": "Bihar",
    "ch": "Chandigarh",
    "ct": "Chattisgarh",
    "dn": "Dadra and Nagar Haveli",
    "dd": "Daman and Diu",
    "dl": "Delhi",
    "ga": "Goa",
    "gj": "Gujarat",
    "hr": "Haryana",
    "hp": "Himachal Pradesh",
    "jk": "Jammu and Kashmir",
    "jh": "Jharkhand",
    "ka": "Karnataka",
    "kl": "Kerala",
    "la": "Ladakh",
    "ld": "Lakshadweep Islands",
    "mp": "Madhya Pradesh",
    "mh": "Maharashtra",
    "mn": "Manipur",
    "ml": "Meghalaya",
    "mz": "Mizoram",
    "nl": "Nagaland",
    "or": "Odisha",
    "ot": "Other Territory",
    "py": "Pondicherry",
    "pb": "Punjab",
    "rj": "Rajasthan",
    "sk": "Sikkim",
    "tn": "Tamil Nadu",
    "tg": "Telangana",
    "tt": "Total",
    "tr": "Tripura",
    "un": "Other",
    "up": "Uttar Pradesh",
    "ut": "Uttarakhand",
    "wb": "West Bengal",
}

In [ ]:
state_total = get_api_data(api_urls["state_total"])
state_total_data = []

for state in state_total:
    if state_codes[state.lower()]:
        state_data = {}
        state_data["state"] = state_codes[state.lower()]
        state_data["confirmed"] = state_total[state]["total"].get("confirmed", 0)
        state_data["deceased"] = state_total[state]["total"].get("deceased", 0)
        state_data["recovered"] = state_total[state]["total"].get("recovered", 0)
        state_data["tested"] = state_total[state]["total"].get("tested", 0)
        state_total_data.append(state_data)

# new schema for spark dataframe
data_schema = [
    StructField("state", StringType(), True),
    StructField("confirmed", IntegerType(), True),
    StructField("deceased", IntegerType(), True),
    StructField("recovered", IntegerType(), True),
    StructField("tested", IntegerType(), True),
]
final_struct = StructType(fields=data_schema)

# convert dict to spark dataframe
total_state_data_df = spark.createDataFrame(state_total_data, final_struct)

Pie chart giving an overview of the entire dataset, showing division of tested patients between different categories


In [ ]:
total_india_data_df = total_state_data_df.filter("state == 'Total'")
total_state_data_df = total_state_data_df.withColumn(
    "unconfirmed", (total_state_data_df["tested"] - total_state_data_df["confirmed"])
)
total_state_data_df = total_state_data_df.withColumn(
    "active",
    (
        total_state_data_df["confirmed"]
        - F.abs(total_state_data_df["deceased"] - total_state_data_df["recovered"])
    ),
)
total_state_data_pd = total_state_data_df.toPandas()

areas = ["recovered", "deceased", "active", None]
total_wise = ["confirmed", "confirmed", "confirmed", "unconfirmed"]
total = ["Tested", "Tested", "Tested", "Tested"]
values = [
    sum(total_state_data_pd["recovered"].astype(int)),
    sum(total_state_data_pd["deceased"].astype(int)),
    sum(total_state_data_pd["active"].astype(int)),
    sum(total_state_data_pd["unconfirmed"].astype(int)),
]
df = pd.DataFrame(
    {"total": total, "total_wise": total_wise, "areas": areas, "values": values}
).reset_index()

fig = px.sunburst(df, path=["total", "total_wise", "areas"], values="values")
fig.update_layout(margin=dict(t=20, l=20, r=20, b=20))
fig.show(rendered="colab")

Analysis:
- The first plot shows that the tests are being conducted at a faster rate.
- The second plot shows that the recovery rate is very high as about 85% of confirmed cases have recovered while those deceased ~5%.


State Wise division of the dataset with Death Rate and Recovery Rate. The data is ordered in descending order with respect to ‘confirmed’.  Here,
- ‘deceased’ + ‘recovered’ + ‘active’ = ‘confirmed’
- ‘confirmed’ + ‘unconfirmed’ = ‘tested’


In [ ]:
total_state_data = (
    total_state_data_df.filter("state != 'Total'")
    .orderBy(total_state_data_df["confirmed"].desc())
    .toPandas()
)
total_state_data["active"] = total_state_data["confirmed"] - abs(
    total_state_data["deceased"] - total_state_data["recovered"]
)
total_state_data["Death Rate"] = np.round(
    100 * total_state_data["deceased"] / total_state_data["confirmed"], 2
)
total_state_data["Recovery Rate"] = np.round(
    100 * total_state_data["recovered"] / total_state_data["confirmed"], 2
)

total_state_data[:15].sort_values("confirmed", ascending=False).fillna(0)\
    .style.background_gradient(cmap="pink", subset=["tested"])\
    .background_gradient(cmap="Reds", subset=["confirmed"])\
    .background_gradient(cmap="Greys", subset=["deceased"])\
    .background_gradient(cmap="Greens", subset=["recovered"])\
    .background_gradient(cmap="Purples", subset=["active"])\
    .background_gradient(cmap="Greys", subset=["death_rate"])\
    .background_gradient(cmap="Oranges", subset=["recovery_rate"])

,state,confirmed,deceased,recovered,tested,unconfirmed,active,Death Rate,Recovery Rate
0,Maharashtra,1683775,44024,1514079,9024871,7341096,213720,2.610000,89.920000
1,Karnataka,827064,11192,765261,8012641,7185577,72995,1.350000,92.530000
2,Andhra Pradesh,825966,6706,795592,8117685,7291719,37080,0.810000,96.320000
3,Tamil Nadu,727026,11152,694880,10029222,9302196,43298,1.530000,95.580000
4,Uttar Pradesh,483832,7051,453458,15013388,14529556,37425,1.460000,93.720000
5,Kerala,440131,1513,348835,4695059,4254928,92809,0.340000,79.260000
6,Delhi,392370,6562,351635,4725318,4332948,47297,1.670000,89.620000
7,West Bengal,377651,6900,333990,4600882,4223231,50561,1.830000,88.440000
8,Odisha,293214,1393,277564,4645192,4351978,17043,0.480000,94.660000
9,Telangana,240048,1341,220466,4323666,4083618,20923,0.560000,91.840000


Analysis:
- The number of confirmed cases are highest in Maharashtra. The confirmed cases are very high in most of the southern states of India than the northern states.
- The number of deceased is also maximum in Maharashtra. The number of deceased is higher in the southern states than the northern states, but the difference is small.
- The number of recovered is quite high across all states as with respect to their confirmed cases.
- The number of unconfirmed cases is high across all states, which was also evident from the previous plots.
- The number of active cases is highest in Maharashtra which exceeds twice that of the second highest.
- The death rate is high for the states of Maharashtra and Gujarat.
- The recovery rate is quite high across all states exceeding 90% for most of the states and the lowest being 79.26% for the state of Kerala.


In [ ]:
state_daily = get_api_data(api_urls["state_daily"])
state_daily_data = json.dumps(state_daily["states_daily"])

In [ ]:
df = spark.read.json(spark.sparkContext.parallelize([state_daily_data]))
df = df.withColumn("date", F.to_date(df.date, "dd-MMM-yy"))
int_cols = []

for col_name in df.columns:
    if col_name not in ["dateymd", "date", "status"]:
        int_cols.append(state_codes[col_name])
        df = df.withColumn(state_codes[col_name], F.col(col_name).cast("int"))
        df = df.drop(col_name)

Date wise count of ‘confirmed’, ‘recovered’ and ‘deceased’ starting 14 March, 2020. This visualization shows the rate at which COVID-19 spread and how the recovery rate was as compared to the detection rate.

In [ ]:
df_day_count = df.select("date", "status", "Total")
fig = px.line(
    df_day_count.toPandas(),
    x="date",
    y="Total",
    hover_data=["status"],
    color="status",
    labels={"Total": "Total count", "date": "Date"},
    height=400,
    title="Daily Count",
)
fig.show(renderer="colab")

Analysis:
- The cases start rising from the middle of April 2020. There is a steady rise in the confirmed and recovered cases towards the middle of July 2020 after which the rate increases slightly, peaking in the month of September. The number of recovered cases exceed the confirmed cases towards the end of September, post which, there is a steady decline with a few spikes. The number of deceased remains low throughout.


Date wise count of COVID-19 detections of the top 15 worst hit states.


In [ ]:
top_states_list = total_state_data["state"][:15]
df_confirmed = df.filter("status='Confirmed'").toPandas()

fig = make_subplots(rows=4, cols=5, subplot_titles=top_states_list)
j, k = 1, 1
for i, state in enumerate(top_states_list):
    fig.add_trace(
        go.Scatter(x=df_confirmed["date"], y=df_confirmed[state]), row=j, col=k
    )
    k = k + 1
    if k > 5:
        k, j = 1, j + 1

fig.update_layout(
    height=900,
    width=900,
    title_text="Confirmed cases of Top 15 States",
    showlegend=False,
)
fig.show(renderer="colab")

Analysis:

| State          | Rise in the number of cases                                    | Peak in the number of cases                                          | Decline in the number of cases                            | Other Observations                                                                                                    |
|----------------|----------------------------------------------------------------|----------------------------------------------------------------------|-----------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------|
| Maharashtra    | Steady rise from April                                         | In the month of September                                            | Relatively sharper decline from mid September             | Steady rise and decline                                                                                               |
| Karnataka      | Relatively rapid rise from July                                | In the month of October                                              | Sharp decline from mid October                            | Few fluctuations during the rise of cases                                                                             |
| Andhra Pradesh | Sharpest spike among all states in July                        | In the month of August and the month of September                    | Steady decline from September                             | Sharpest spike, cases peaked twice                                                                                    |
| Tamil Nadu     | Steady rise from May                                           | In the month of August                                               | Slow decline from August with steady decline from October | Steady rise and decline                                                                                               |
| Uttar Pradesh  | Relatively rapid rise from July                                | In the month of September                                            | Relatively sharper decline from mid September             | Steady rise and decline                                                                                               |
| Kerala         | Slow rise from June, with relatively sharp rise in September   | In the month of October                                              | Steady decline from October                               | Few fluctuations during decline of cases                                                                              |
| Delhi          | Steady rise May and September, sharp spike in October          | In June, September and November                                      | Steady decline in July and September                      | Large fluctuations during rise and decline in number of cases, currently highest number of cases w.r.t the population |
| West Bengal    | Steady rise from May, with relatively high rate in July        | Towards end of October                                               | Steady decline from November                              | Currently one of the highest number of cases w.r.t the population                                                     |
| Odisha         | Steady rise with relatively higher rate from July              | Towards end of September                                             | Relatively sharper decline from October                   | Steady rise and decline                                                                                               |
| Telangana      | Steady rise from June                                          | Towards end of August                                                | Steady decline from September                             | Large fluctuations during rise and decline of cases. Sudden drop in number of cases in August                         |
| Bihar          | Steady rise from May, with relatively very high rate from July | In the month of August                                               | Steady decline from Mid August                            | Few fluctuations during decline of cases                                                                              |
| Assam          | Steady rise from May                                           | In the month of August, with smaller spikes in September and October | Steady decline with a few spikes from Mid August          | Large fluctuations during rise and decline of cases.                                                                  |
| Rajasthan      | Steady rise from May                                           | In the month of October                                              | Steady decline from mid October                           | Steady rise and decline                                                                                               |
| Chhattisgarh   | Sharp spike in August                                          | In the month of October                                              | Steady decline from mid October                           | Few fluctuations during decline of  cases                                                                             |
| Gujarat        | Steady rise from April, with a spike in mid May                | In the month of October                                              | Steady decline from mid October                           | Steady rise and decline, with sudden spike in mid May                                                                 |


Date wise count of COVID-19 detections of all Indian states. This visualization enables comparative study between COVID-19 case count of different states.


In [ ]:
exprs = {x: "sum" for x in int_cols}
year_month_df = (
    df.groupby(F.date_format("date", "yyyy-MM").alias("year_month_sum"))
    .agg(exprs)
    .orderBy(F.month("year_month_sum"))
)

for column in year_month_df.columns:
    start_index = column.find("(")
    end_index = column.find(")")
    if start_index and end_index and column != "year_month_sum":
        year_month_df = year_month_df.withColumnRenamed(
            column, column[start_index + 1 : end_index]
        )
        year_month_df = year_month_df.drop(column)

year_month_dft = year_month_df.select(
    [c for c in year_month_df.columns if c not in {"Total"}]
)
new_year_month_df = to_long(year_month_dft, ["year_month_sum"])

fig = px.line(
    new_year_month_df.toPandas(),
    x="year_month_sum",
    y="val",
    color="key",
    title="Monthly report of Indiain states",
    labels={"year_month_sum": "Month", "key": "State", "val": "Total Count"},
)
fig.show(renderer="colab")

Analysis:
- India witnessed a COVID-19 outbreak in the month of February, with the number of cases increasing rapidly from March and April in most of the States.
- Nation-wide lockdown imposed in the month of March, controlled the number of cases in most of the States and enabled a significant growth in testing.
- The strict lockdown extended till early June, during which the rise in the cases was steady. With the beginning of ‘unlocking’ of the lockdown, many states showed a rapid rise in the number of cases, with major cities like Mumbai, Delhi, Chennai and other few accounting for ~50% of total reported cases in the country.
- In the month of July, India’s fatality rate was the lowest in the world and steadily declining.
- Infection rates started to drop significantly in September, and the number of daily new cases and active cases started to decline rapidly and recoveries exceeded the active cases.
- COVID-19 confirmed cases peaked in October in many States.
- Currently, Lakshadweep is the only region which has not reported a single case.
